In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install Cython

Defaulting to user installation because normal site-packages is not writeable


In [7]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""


## Install dependencies
!pip install wget
!pip install faiss-gpu

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [8]:

## Install NeMo
BRANCH = 'v1.17.0'
#!python3 -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
!cd /mnt/d/school/clef2023/sapbert/nemo/NeMo-1.17.0/NeMo-1.17.0 && python3 -m pip install .


Defaulting to user installation because normal site-packages is not writeable
Processing /mnt/d/school/clef2023/sapbert/nemo/NeMo-1.17.0/NeMo-1.17.0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


  Created wheel for nemo-toolkit: filename=nemo_toolkit-1.17.0-py3-none-any.whl size=2255023 sha256=e848a5ddae8330eb938eb3ec83aa5eaeeb12a81e16e6cd5a34fbdee59a61735e
  Stored in directory: /home/sylvia/.cache/pip/wheels/22/f1/15/59b45677bb3780b113715bac3be48c2a0c179700bf11ac2214
Successfully built nemo-toolkit
  Attempting uninstall: nemo-toolkit
    Found existing installation: nemo-toolkit 1.17.0
    Uninstalling nemo-toolkit-1.17.0:
      Successfully uninstalled nemo-toolkit-1.17.0


In [2]:
from nemo.collections.nlp.modules.common.huggingface.xlmroberta import XLMRobertaEncoder

[NeMo W 2023-09-30 16:17:30 nemo_logging:349] /home/sylvia/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
      warnings.warn(
    
[NeMo W 2023-09-30 16:17:30 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-09-30 16:17:30 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-09-30 16:17:30 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-09-30 16:17:31 nemo_logging:349] /home/sylvia/.local/lib/python3.10/site-packages/pydub/ut

In [3]:
import faiss
import torch
import wget
import os
import numpy as np
import pandas as pd

from omegaconf import OmegaConf
from pytorch_lightning import Trainer
from IPython.display import display
from tqdm import tqdm

from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

## Entity Linking

#### Task Description
[Entity linking](https://en.wikipedia.org/wiki/Entity_linking) is the process of connecting concepts mentioned in natural language to their canonical forms stored in a knowledge base. For example, say a knowledge base contained the entity 'ID3452 influenza' and we wanted to process some natural language containing the sentence "The patient has flu like symptoms". An entity linking model would match the word 'flu' to the knowledge base entity 'ID3452 influenza', allowing for disambiguation and normalization of concepts referenced in text. Entity linking applications range from helping automate data ingestion to assisting in real time dialogue concept normalization. We will be focusing on entity linking in the medical domain for this demo, but the entity linking model, dataset, and training code within NVIDIA NeMo can be applied to other domains like finance and retail.

Within NeMo and this tutorial we use the entity linking approach described in Liu et. al's NAACL 2021 "[Self-alignment Pre-training for Biomedical Entity Representations](https://arxiv.org/abs/2010.11784v2)". The main idea behind this approach is to reshape an initial concept embedding space such that synonyms of the same concept are pulled closer together and unrelated concepts are pushed further apart. The concept embeddings from this reshaped space can then be used to build a knowledge base embedding index. This index stores concept IDs mapped to their respective concept embeddings in a format conducive to efficient nearest neighbor search. We can link query concepts to their canonical forms in the knowledge base by performing a nearest neighbor search- matching concept query embeddings to the most similar concepts embeddings in the knowledge base index. 

In this tutorial we will be using the [faiss](https://github.com/facebookresearch/faiss) library to build our concept index.

#### Self Alignment Pretraining
Self-Alignment pretraining is a second stage pretraining of an existing encoder (called second stage because the encoder model can be further finetuned after this more general pretraining step). The dataset used during training consists of pairs of concept synonyms that map to the same ID. At each training iteration, we only select *hard* examples present in the mini batch to calculate the loss and update the model weights. In this context, a hard example is an example where a concept is closer to an unrelated concept in the mini batch than it is to the synonym concept it is paired with by some margin. I encourage you to take a look at [section 2 of the paper](https://arxiv.org/pdf/2010.11784.pdf) for a more formal and in depth description of how hard examples are selected.

We then use a [metric learning loss](https://openaccess.thecvf.com/content_CVPR_2019/papers/Wang_Multi-Similarity_Loss_With_General_Pair_Weighting_for_Deep_Metric_Learning_CVPR_2019_paper.pdf) calculated from the hard examples selected. This loss helps reshape the embedding space. The concept representation space is rearranged to be more suitable for entity matching via embedding cosine similarity. 

Now that we have idea of what's going on, let's get started!

## Dataset Preprocessing

In [4]:
# Download data into project directory
PROJECT_DIR = "." #"/content/drive/MyDrive/sapbert/" #Change if you don't want the current directory to be the project dir
DATA_DIR = "." #os.path.join(PROJECT_DIR, "medprocner_data")

#if not os.path.isdir(os.path.join(DATA_DIR)):
    #wget.download('https://dldata-public.s3.us-east-2.amazonaws.com/tiny_example_data.zip', os.path.join(PROJECT_DIR, "tiny_example_data.zip"))

 #   !unzip {PROJECT_DIR}/tiny_example_data.zip -d {PROJECT_DIR}

In this tutorial we will be using a tiny toy dataset to demonstrate how to use NeMo's entity linking model functionality. The dataset includes synonyms for 12 medical concepts. Entity phrases with the same ID are synonyms for the same concept. For example, "*chronic kidney failure*", "*gradual loss of kidney function*", and "*CKD*" are all synonyms of concept ID 5. Here's the dataset before preprocessing:

In [23]:
#raw_data = pd.read_csv(os.path.join(DATA_DIR, "tiny_example_dev_data.csv"), names=["ID", "CONCEPT"], index_col=False)
#print(raw_data)

We've already paired off the concepts for this dataset with the format `ID concept_synonym1 concept_synonym2`. Here are the first ten rows:

In [24]:
#training_data = pd.read_table(os.path.join(DATA_DIR, "tiny_example_train_pairs.tsv"), names=["ID", "CONCEPT_SYN1", "CONCEPT_SYN2"], delimiter='\t')
#print(training_data.head(10))

Use the [Unified Medical Language System (UMLS)](https://www.nlm.nih.gov/research/umls/index.html) dataset for full medical domain entity linking training. The data contains over 9 million entities and is a table of medical concepts with their corresponding concept IDs (CUI). After [requesting a free license and making a UMLS Terminology Services (UTS) account](https://www.nlm.nih.gov/research/umls/index.html), the [entire UMLS dataset](https://www.nlm.nih.gov/research/umls/licensedcontent/umlsknowledgesources.html) can be downloaded from the NIH's website. If you've cloned the NeMo repo you can run the data processing script located in `examples/nlp/entity_linking/data/umls_dataset_processing.py` on the full dataset. This script will take in the initial table of UMLS concepts and produce a .tsv file with each row formatted as `CUI\tconcept_synonym1\tconcept_synonym2`. Once the UMLS dataset .RRF file is downloaded, the script can be run from the `examples/nlp/entity_linking` directory like so: 
```
python data/umls_dataset_processing.py
```

## Model Training

Second stage pretrain a BERT Base encoder on the self-alignment pretraining task (SAP) for improved entity linking. Using a GPU, the model should take 5 minutes or less to train on this example dataset and training progress will be output below the cell.

In [25]:
!pip install --upgrade huggingface_hub

#!huggingface-cli login 

Defaulting to user installation because normal site-packages is not writeable
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/sylvia/.cache/huggingface/token
Login successful


In [26]:
#from transformers import AutoTokenizer, AutoModel

#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

#model = AutoModel.from_pretrained("cambridgeltl/SapBERT-UMLS-2020AB-all-lang-from-XLMR")

In [27]:
# Download config
#wget.download("https://github.com/NVIDIA/NeMo/blob/v1.17.0/examples/nlp/entity_linking/conf/tiny_example_entity_linking_config.yaml",
#              os.path.join(PROJECT_DIR, "tiny_example_entity_linking_config.yaml"))

# Load in config file
#cfg = OmegaConf.load(os.path.join(PROJECT_DIR, "xlmr-sapbert-large_entity_linking_config.yaml"))

# Set config file variables
#cfg.project_dir = PROJECT_DIR
#cfg.model.nemo_path = os.path.join(PROJECT_DIR, "xlmr_sap_bert_large_model.nemo")
#cfg.model.train_ds.data_file = os.path.join(DATA_DIR, "tiny_example_train_pairs.tsv") # todo: update
#cfg.model.validation_ds.data_file = os.path.join(DATA_DIR, "tiny_example_validation_pairs.tsv") # todo: update

In [28]:
#if 'checkpoint_callback' in cfg.trainer.keys():
#  cfg.trainer.pop('checkpoint_callback')

#cfg.trainer['accelerator'] = 'cuda'
#cfg.trainer['max_steps'] = -1
#cfg.exp_manager['create_checkpoint_callback'] = False

#MisconfigurationException: `Trainer(strategy='ddp')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: `Fabric(strategy=None|'dp'|'ddp_notebook')`. In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.
#cfg.trainer['strategy'] = 'dp'
#cfg.trainer

In [29]:
# Initialize the trainer and model
#trainer = Trainer(**cfg.trainer)
#exp_manager(trainer, cfg.get("exp_manager", None))
#model = nemo_nlp.models.EntityLinkingModel(cfg=cfg.model, trainer=trainer)

In [30]:
# Train and save the model
#trainer.fit(model)

In [31]:
#model.save_to(cfg.model.nemo_path)

You can run the script at `examples/nlp/entity_linking/self_alignment_pretraining.py` to train a model on a larger dataset. Run

```
python self_alignment_pretraining.py project_dir=.
```
from the `examples/nlp/entity_linking` directory.

## Model Evaluation

Let's evaluate our freshly trained model and compare its performance with a BERT Base encoder that hasn't undergone self-alignment pretraining. We first need to restore our trained model and load our BERT Base Baseline model.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Load original model
base_model_cfg = OmegaConf.load(os.path.join(PROJECT_DIR, "sapbert-large_entity_linking_config.yaml"))
#sapbert-large_entity_linking_config.yaml

# Set train/val datasets to None to avoid loading datasets associated with training
base_model_cfg.model.train_ds = None
base_model_cfg.model.validation_ds = None
#base_model_cfg.index.index_save_name = os.path.join(PROJECT_DIR, "base_model_index")
#base_model_cfg.index.index_ds.data_file = os.path.join(DATA_DIR, "tiny_example_index_data.tsv")
base_model = nemo_nlp.models.EntityLinkingModel(base_model_cfg.model).to(device)


[NeMo W 2023-09-30 16:17:42 model_utils:420] Skipped conversion for config/subconfig:
    {'nemo_path': '???', 'max_seq_length': 128, 'language_model': {'pretrained_model_name': 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext', 'config_file': None, 'config': None, 'lm_checkpoint': None}, 'tokenizer': {'tokenizer_name': 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext', 'vocab_file': 'sapbert/vocab.txt', 'tokenizer_model': None, 'do_lower_case': True}, 'loss_params': None, 'train_ds': None, 'validation_ds': None, 'optim': {'name': 'adam', 'lr': 3e-05, 'weight_decay': 0.0, 'sched': {'name': 'CosineAnnealing', 'warmup_steps': None, 'warmup_ratio': 0.1, 'min_lr': 0.0, 'last_epoch': -1}}}
     Reason: Missing mandatory value: nemo_path
        full_key: nemo_path
        object_type=dict.
[NeMo W 2023-09-30 16:17:42 nlp_overrides:230] Apex was not found. Please see the NeMo README for installation instructions: https://github.com/NVIDIA/apex
    Megatron-based models require Apex to function c

In [34]:
# Restore second stage pretrained model
#sap_model_cfg = OmegaConf.load(os.path.join(PROJECT_DIR, "xlmr-sapbert-large_entity_linking_config.yaml"))
#sap_model_cfg = cfg
#sap_model_cfg.index.index_save_name = os.path.join(PROJECT_DIR, "tiny_example_entity_linking_index")
#sap_model_cfg.index.index_ds.data_file = os.path.join(DATA_DIR, "tiny_example_index_data.tsv")
#cfg.project_dir = PROJECT_DIR
#sap_model_cfg.model.nemo_path = os.path.join(PROJECT_DIR, "tiny_example_sap_bert_model.nemo")
#sap_model = nemo_nlp.models.EntityLinkingModel.restore_from(sap_model_cfg.model.nemo_path).to(device)

We are going evaluate our model on a nearest neighbor task using top 1 and top 5 accuracies as our metric. We will be using a tiny example test knowledge base and test queries. For this evaluation we are going to be comparing every test query with every concept vector in our test set knowledge base. We will rank each item in the knowledge base by its cosine similarity with the test query. We'll then compare the IDs of the predicted most similar test knowledge base concepts with our ground truth query IDs to calculate top 1 and top 5 accuracies. For this metric higher is better.

In [7]:
# Helper function to get data embeddings
def get_embeddings(model, dataloader, filename='kb', model_folder=''): #xlmr/
    embeddings, cids, concept_texts = [], [], []
    batch_size = 64
    batch_idx = 0
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, token_type_ids, attention_mask, batch_cids = batch
                
            batch_embeddings = model.forward(input_ids=input_ids.to(device), 
                                             token_type_ids=token_type_ids.to(device), 
                                             attention_mask=attention_mask.to(device))
            
            # Accumulate index embeddings and their corresponding IDs
            embeddings.extend(batch_embeddings.cpu().detach().numpy())
            
            cids.extend(batch_cids.cpu().detach().numpy())
            
            #print(input_ids.shape)
            for idx in range(input_ids.shape[0]):
                concept = model.tokenizer.decode(input_ids[idx], skip_special_tokens=True)
                concept_texts.append(concept)
                
            batch_idx += 1
            if batch_idx % 1000 == 0:                
                with open(f'files/{model_folder}umls_{filename}_{batch_idx}.npy', 'wb') as f:
                    np.save(f, np.array(embeddings))                
                with open(f'files/{model_folder}umls_{filename}_cids_{batch_idx}.npy', 'wb') as f:
                    np.save(f, cids)
                with open(f'files/{model_folder}umls_{filename}_texts_{batch_idx}.npy', 'wb') as f:
                    np.save(f, concept_texts)
                embeddings, cids, concept_texts = [], [], []
     
    if batch_idx % 1000 != 0:
        with open(f'files/{model_folder}umls_{filename}_{batch_idx}.npy', 'wb') as f:
            np.save(f, np.array(embeddings))                
        with open(f'files/{model_folder}umls_{filename}_cids_{batch_idx}.npy', 'wb') as f:
            np.save(f, cids)
        with open(f'files/{model_folder}umls_{filename}_texts_{batch_idx}.npy', 'wb') as f:
            np.save(f, concept_texts)
    
    return embeddings, cids, concept_texts

In [36]:
def get_query_embedding(queries, model):
    # Tokenize our queries
    model_input =  model.tokenizer(queries,
                                   add_special_tokens = True,
                                   padding = True,
                                   truncation = True,
                                   max_length = 512,
                                   return_token_type_ids = True,
                                   return_attention_mask = True)
    
    # Pass tokenized input into model
    query_emb =  model.forward(input_ids=torch.LongTensor(model_input["input_ids"]).to(device),
                               token_type_ids=torch.LongTensor(model_input["token_type_ids"]).to(device),
                               attention_mask=torch.LongTensor(model_input["attention_mask"]).to(device))
    
    return query_emb

In [37]:
def evaluate(model, test_kb_embs, test_kb_cids, test_queries, ks):
    # Initialize knowledge base and query data loaders
    results_top = {}
    
    for k in ks:
        results_top[k] = []
    
    #test_kb_dataloader = model.setup_dataloader(test_kb, is_index_data=True)
    test_query_dataloader = model.setup_dataloader(test_queries, is_index_data=True)
    
    # Get knowledge base and query embeddings
    #test_kb_embs, test_kb_cids = get_embeddings(model, test_kb_dataloader)
    test_query_embs, test_query_cids, concepts = get_embeddings(model, test_query_dataloader)

    # Calculate the cosine distance between each query and knowledge base concept
    score_matrix = np.matmul(np.array(test_query_embs), np.array(test_kb_embs).T)
    accs = {k : 0 for k in ks}
    
    # Compare the knowledge base IDs of the knowledge base entities with 
    # the smallest cosine distance from the query 
    for query_idx in tqdm(range(len(test_query_cids))):
        query_emb = test_query_embs[query_idx]
        query_cid = test_query_cids[query_idx]
        query_scores = score_matrix[query_idx]

        for k in ks:
            topk_idxs = np.argpartition(query_scores, -k)[-k:]
            topk_cids = [test_kb_cids[idx] for idx in topk_idxs]
            
            results_top[k].append(topk_cids)
            
            # If the correct query ID is amoung the top k closest kb IDs
            # the model correctly linked the entity
            match = int(query_cid in topk_cids)
            accs[k] += match

    for k in ks:
        accs[k] /= len(test_query_cids)
                
    return accs, results_top

In [38]:
def get_kb_embs(model, test_kb):
    test_kb_dataloader = model.setup_dataloader(test_kb, is_index_data=True)
    #test_query_dataloader = model.setup_dataloader(test_queries, is_index_data=True)
    
    # Get knowledge base and query embeddings
    test_kb_embs, test_kb_cids, concepts = get_embeddings(model, test_kb_dataloader)
    
    return test_kb_embs, test_kb_cids, concepts

In [39]:
def predict(model, test_kb_embs, test_kb_cids, queries, k):
    # Initialize knowledge base and query data loaders
    #results_top_1 = []
    results = []
    result_ids = []
    #test_kb_dataloader = model.setup_dataloader(test_kb, is_index_data=True)
    #test_query_dataloader = model.setup_dataloader(test_queries, is_index_data=True)
    
    # Get knowledge base and query embeddings
    #test_kb_embs, test_kb_cids = get_embeddings(model, test_kb_dataloader)
    #test_query_embs, test_query_cids = get_embeddings(model, test_query_dataloader)
    test_query_embs = get_query_embedding(queries, model).cpu().detach().numpy()

    # Calculate the cosine distance between each query and knowledge base concept
    score_matrix = np.matmul(np.array(test_query_embs), np.array(test_kb_embs).T)
    #accs = {k : 0 for k in ks}
    
    # Compare the knowledge base IDs of the knowledge base entities with 
    # the smallest cosine distance from the query 
    for query_idx in tqdm(range(len(queries))):
        query_emb = test_query_embs[query_idx]
        #query_cid = test_query_cids[query_idx]
        query_scores = score_matrix[query_idx]

        #for k in ks:
        topk_idxs = np.argpartition(query_scores, -k)[-k:]
        topk_cids = [test_kb_cids[idx].item() for idx in topk_idxs]
            
        #    if k == 1:
        results.append(topk_cids)
        result_ids.append(topk_idxs)
            
            # If the correct query ID is amoung the top k closest kb IDs
            # the model correctly linked the entity
         #   match = int(query_cid in topk_cids)
         #   accs[k] += match

    #for k in ks:
    #    accs[k] /= len(test_query_cids)
                
    #return accs, results_top_1
    return results, result_ids

In [40]:
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [41]:
#paper experiments performed with - umls_val_kb, test submission - full_final_kb

In [17]:
kb_file = 'umls_en_kb_idx.tsv' #"umls_procedures_full.tsv", full_final_kb val_kb
query_file = 'mm_llama_umls_query.tsv' #bg_pred_umls_query

In [18]:
import numpy as np

In [19]:
# Create configs for our test data

test_kb = OmegaConf.create({
    "data_file": os.path.join(DATA_DIR, kb_file),
    "max_seq_length": 128,
    "batch_size": 64,
    "shuffle": False,
})

test_queries = OmegaConf.create({
    "data_file": os.path.join(DATA_DIR, query_file),
    "max_seq_length": 128,
    "batch_size": 64,
    "shuffle": False,
})



In [20]:
test_query_dataloader = base_model.setup_dataloader(test_queries, is_index_data=True)    

[NeMo I 2023-09-30 16:37:04 entity_linking_dataset:61] Getting datafile newline indices
[NeMo I 2023-09-30 16:37:04 entity_linking_dataset:78] Loaded dataset with 27446 examples


In [21]:
test_query_embs, test_query_cids, concepts = get_embeddings(base_model, test_query_dataloader, 'mm_pred_test_llama')

100%|█████████████████████████████████████████████████████████████████████| 429/429 [00:31<00:00, 13.84it/s]


In [22]:
test_kb_embs, test_kb_cids, test_kb_concepts = get_kb_embs(base_model, test_kb)

[NeMo I 2023-08-27 14:59:16 entity_linking_dataset:78] Loaded dataset with 8510773 examples


  0%|                                                                                                                     | 0/132981 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  8%|████████▌                                                                                              | 11096/132981 [26:56<3:05:03, 10.98it/s]

In [23]:
#60098 -> 60102
len(test_kb_embs)

In [24]:
batch_idx = 'last'
with open(f'files/xlmr/umls_kb_{batch_idx}.npy', 'wb') as f:
    np.save(f, np.array(test_kb_embs))
with open(f'files/xlmr/umls_kb_cids_{batch_idx}.npy', 'wb') as f:
    np.save(f, test_kb_cids)
with open(f'files/xlmr/umls_kb_texts_{batch_idx}.npy', 'wb') as f:
    np.save(f, test_kb_concepts)

The purpose of this section was to show an example of evaluating your entity linking model. This evaluation set contains very little data, and no serious conclusions should be drawn about model performance. Top 1 accuracy should be between 0.7 and 1.0 for both models and top 5 accuracy should be between 0.8 and 1.0. When evaluating a model trained on a larger dataset, you can use a nearest neighbors index to speed up the evaluation time.

## Building an Index

To qualitatively observe the improvement we gain from the second stage pretraining, let's build two indices. One will be built with BERT base embeddings before self-alignment pretraining and one will be built with the model we just trained. Our knowledge base in this tutorial will be in the same domain and have some overlapping concepts as the training set. This data file is formatted as `ID\tconcept`.

The `EntityLinkingDataset` class can load the data used for training the entity linking encoder as well as for building the index if the `is_index_data` flag is set to true. 

In [18]:
def build_index(cfg, model):
    # Setup index dataset loader
    index_dataloader = model.setup_dataloader(cfg.index.index_ds, is_index_data=True)
    
    # Get index dataset embeddings
    embeddings, _, _ = get_embeddings(model, index_dataloader)
    
    # Train IVFFlat index using faiss
    embeddings = np.array(embeddings)
    quantizer = faiss.IndexFlatL2(cfg.index.dims)
    index = faiss.IndexIVFFlat(quantizer, cfg.index.dims, cfg.index.nlist)
    #index = faiss.index_cpu_to_all_gpus(index)
    #index = faiss.index(index)
    index.train(embeddings)
    
    # Add concept embeddings to index
    for i in tqdm(range(0, embeddings.shape[0], cfg.index.index_batch_size)):
            index.add(embeddings[i:i+cfg.index.index_batch_size])

    # Save index
    #faiss.write_index(faiss.index_gpu_to_cpu(index), cfg.index.index_save_name)
    faiss.write_index(index, cfg.index.index_save_name)

In [21]:
build_index(sap_model_cfg, sap_model.to(device))
#build_index(base_model_cfg, base_model.to(device))

[NeMo I 2023-05-12 22:35:50 entity_linking_dataset:61] Getting datafile newline indices
[NeMo I 2023-05-12 22:35:50 entity_linking_dataset:78] Loaded dataset with 55394 examples


100%|█████████████████████████████████████████████████████████████████| 5540/5540 [00:08<00:00, 664.16it/s]


## Entity Linking via Nearest Neighbor Search

Now it's time to query our indices! We are going to query both our index built with embeddings from BERT Base, and our index with embeddings built from the SAP BERT model we trained. Our sample query phrases will be "*high blood sugar*" and "*head pain*". 

To query our indices, we first need to get the embedding of each query from the corresponding encoder model. We can then pass these query embeddings into the faiss index which will perform a nearest neighbor search, using cosine distance to compare the query embedding with embeddings present in the index. Once we get a list of knowledge base index concept IDs most closely matching our query, all that is left to do is map the IDs to a representative string describing the concept. 

In [33]:
def query_index(cfg, model, index, queries, id2string, id2snomed):
    # Get query embeddings from our entity linking encoder model
    query_embs = get_query_embedding(queries, model).cpu().detach().numpy()
    
    # Use query embedding to find closest concept embedding in knowledge base
    distances, neighbors = index.search(query_embs, 1) #cfg.index.top_n)
    
    # Get the canonical strings corresponding to the IDs of the query's nearest neighbors in the kb 
    neighbor_concepts = [[id2string[concept_id] for concept_id in query_neighbor] \
                                                for query_neighbor in neighbors]
    
    results = []
    # Display most similar concepts in the knowledge base. 
    for query_idx in range(len(queries)):
        #print(f"\nThe most similar concepts to {queries[query_idx]} are:")
        for cid, concept, dist in zip(neighbors[query_idx], neighbor_concepts[query_idx], distances[query_idx]):
            results.append((cid, concept, id2snomed[cid], 1 - dist))
            
    return results

    
def get_query_embedding(queries, model):
    # Tokenize our queries
    model_input =  model.tokenizer(queries,
                                   add_special_tokens = True,
                                   padding = True,
                                   truncation = True,
                                   max_length = 512,
                                   return_token_type_ids = True,
                                   return_attention_mask = True)
    
    # Pass tokenized input into model
    query_emb =  model.forward(input_ids=torch.LongTensor(model_input["input_ids"]).to(device),
                               token_type_ids=torch.LongTensor(model_input["token_type_ids"]).to(device),
                               attention_mask=torch.LongTensor(model_input["attention_mask"]).to(device))
    
    return query_emb

In [34]:
# Load indices
sap_index = faiss.read_index(sap_model_cfg.index.index_save_name)
#base_index = faiss.read_index(base_model_cfg.index.index_save_name)

In [35]:
# Map concept IDs to one canonical string
index_data = open(sap_model_cfg.index.index_ds.data_file, "r", encoding='utf-8-sig')
id2string = {}
id2snomed = {}

idx = 0
for line in index_data:
    cid, concept = line.split("\t")
    #id2string[int(cid) - 1] = concept.strip()
    id2string[idx] = concept.strip()
    id2snomed[idx] = cid

    idx += 1

In [36]:
len(id2string.keys())

55394

In [37]:
id2string[3]

'ínyección terapéutica de muñeca derecha con guía fluoroscópica'

In [38]:
print(id2snomed[0], id2snomed[1])

16555451000119104 16549151000119108


In [39]:
# Some sample queries
queries = ['evaluar los efectos colaterales de la medicación'] #id2string[0], id2string[1]]
#["Se monitorizó de manera seriada el peso del producto de la gestación", "remoción de un lentículo", "secuestrectomía vertebral"]

# Query BERT Base
#print("BERT Base output before Self Alignment Pretraining:")
#query_index(base_model_cfg, base_model, base_index, queries, id2string)
#print("\n" + "-" * 50 + "\n")

# Query SAP BERT
print("SAP BERT output after Self Alignment Pretraining:")
results = query_index(sap_model_cfg, sap_model, sap_index, queries, id2string, id2snomed)

for result in results:
    print(result)
print("\n" + "-" * 50 + "\n")

SAP BERT output after Self Alignment Pretraining:
(35664, 'evaluación de los efectos colaterales de la medicación', '1177155004', 0.3694462180137634)

--------------------------------------------------



Even after only training on this tiny amount of data, the qualitative performance boost from self-alignment pretraining is visible. The baseline model links "*high blood sugar*" to the entity "*6 diabetes*" while our SAP BERT model accurately links "*high blood sugar*" to "*Hyperinsulinemia*". Similarly, "*head pain*" and "*Myocardial infraction*" are not the same concept, but "*head pain*" and "*Headache*" are.

For larger knowledge bases keeping the default embedding size might be too large and cause out of memory issues. You can apply PCA or some other dimensionality reduction method to your data to reduce its memory footprint. Code for creating a text file of all the UMLS entities in the correct format needed to build an index and creating a dictionary mapping concept ids to canonical concept strings can be found here `examples/nlp/entity_linking/data/umls_dataset_processing.py`. 

The code for extracting knowledge base concept embeddings, training and applying a PCA transformation to the embeddings, building a faiss index and querying the index from the command line is located at `examples/nlp/entity_linking/build_index.py` and `examples/nlp/entity_linking/query_index.py`. 

If you've cloned the NeMo repo, both of these steps can be run as follows on the command line from the `examples/nlp/entity_linking/` directory.

```
python data/umls_dataset_processing.py --index
python build_index.py --restore
python query_index.py --restore
```
By default the project directory will be "." but can be changed by adding the flag `--project_dir=<PATH>` after each of the above commands. Intermediate steps of the index building process are saved. In the occurrence of an error, previously completed steps do not need to be rerun. 

## Command Recap

Here is a recap of the commands and steps to repeat this process on the full UMLS dataset. 

1) Download the UMLS dataset file `MRCONSO.RRF` from the NIH website and place it in the `examples/nlp/entity_linking/data` directory.

2) Run the following commands from the `examples/nlp/entity_linking` directory
```
python data/umls_dataset_processing.py
python self_alignment_pretraining.py project_dir=. 
python data/umls_dataset_processing.py --index
python build_index.py --restore
python query_index.py --restore
```
The model will take ~24hrs to train on two GPUs and ~48hrs to train on one GPU. By default the project directory will be "." but can be changed by adding the flag `--project_dir=<PATH>` after each of the above commands and changing `project_dir=<PATH>` in the `self_alignment_pretraining.py` command. If you change the project directory, you should also move the `MRCONOSO.RRF` file to a `data` sub directory within the one you've specified. 

As mentioned in the introduction, entity linking within NVIDIA NeMo is not limited to the medical domain. The same data processing and training steps can be applied to a variety of domains and use cases. You can edit the datasets used as well as training and loss function hyperparameters within your config file to better suit your domain.